# Analytics Notebook  
Goal is to find the patterns of the problem given the examples we were provided.  


## Test Cases

This is a list of all of the problem properties that were given by the Philly team.  We can use these to build out the code and test it



In [1]:
%%sql
SELECT *
FROM davelake.philly_problempropertylist

StatementMeta(, a9932df5-ba26-4545-969a-1262f372872d, 2, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 2 fields>

## philly_flattened_properties table

* 1 entry for every property
* will have all of the `features` (attributes, properties) for every property
* will have a score between 0-1 for each `factor`, named like `F1_Repairs`
* in subsequent cells we'll build out the code to calculate each of these factors
* we can use this table to join to other tables, as the source of ML to predict future cases, testing, dashboard

Start by seeding the table with every property and base details

In [2]:
%%sql 

--driving table
CREATE OR REPLACE TABLE davelake.philly_flattened_properties
AS 
select 
    opa.parcel_number,
    opa.location,
    opa.owner_1,
    opa.owner_2,
    problems.MaggyReason as MaggyLabel,
    0 AS F1_Repairs,
    0 AS F2_CodeViolations,
    0 AS F3_TaxesAndDebt,
    0 AS F4_CityResources,
    0 AS F5_Licenses,
    0 AS F6_CorpStructures,
    0 AS F7_Nuisances,
    0 AS F8_Insurance,
    0 AS F9_Zoning,
    0 AS F10_SheriffSalePurchase,
    0 AS F11_AvoidSquatters,
    0 AS F12_LengthOfOwnership,
    0 AS F13_HomeValueAppreciation,
    0 AS F14_BusinessesInArea,
    0 AS F15_NeighborhoodVacantLots,
    0 AS F16_LocalInvestors,
    0 AS F17_MultipleHomesteadExemptions,
    CASE opa.homestead_exemption WHEN 0 THEN 0 ELSE 1 END AS F18_OwnerOccupied,
    0 AS F19_DollarTransfers,
    opa.homestead_exemption,
    opa.zoning
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_problempropertylist problems
    on opa.location = problems.location
;

StatementMeta(, 71f05189-1c4e-44ba-a0b2-147011c87d65, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%sql

select * from davelake.philly_flattened_properties
limit 10

StatementMeta(, 71f05189-1c4e-44ba-a0b2-147011c87d65, 4, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 26 fields>

## Build out the _Factors_

Factors that determine this, with a code.  We'll build out the code in order in subsequent cells and explain in detail the calculations there.  

* `F1_Repairs`:  Fail or refuse to repair or invest in properties  
* `F2_CodeViolations`:  Tend to incur code violations 
* `F3_TaxesAndDebt`: Fail to pay Real Estate taxes and/or other city debt such as water bills and code enforcement abatement bills  
* `F4_CityResources`: Use city resources to maintain property.  
    * mowing the lawn
    * sealing the property
    * demolishing it 
* `F5_Licenses`: Fail to maintain licenses. 
    * vacant property license (non-occupied property) 
    * rental license (occupied property)
* `F6_CorpStructures`: Attempt to shield their true identity via complicated corporate structures 
* `F7_Nuisances`: Nuisances within the neighborhood.  Either by virtue of how the property is maintained and/or the scope of the properties that they have, that might have minor problems that cumulatively become problematic. 
* `F8_Insurance`: not insuring the property 
* `F9_Zoning`: not using the property for what it is zoned for 
* `F10_SheriffSalePurchase`: buying the property at sheriff's sale and waiting for the neighborhood to turn around.  
* `F11_AvoidSquatters`: actively trying to avoid squatters 
* `F12_LengthOfOwnership`: length of ownership 
* `F13_HomeValueAppreciation`: home value appreciation 
  * TODO:  from 8/19.  Might need to review this one with Philly team.  I don't understand the factor.  
  * investor strips all value out of it and not putting anything into it at all and have no intention of doing that.  How would we detect/know this?  
  * or is it a case where the property is bought in anticipation of rising property values in the area and when that doesn't happen it is just abandoned? 
* `F14_BusinessesInArea`: varieties of businesses in the area (grocery stores vs bodegas/papis) 
  * vaping stores on every block (drug paraphrenalia)
* `F15_NeighborhoodVacantLots`: number of vacant lots in neighborhood 
* `F16_LocalInvestors`: in-town vs out-of-town investors (this has been proven to NOT be the latter).  
* `F17_MultipleHomesteadExemptions`: Multiple Homestead Exemptions 
  * if an owner has 2 homestead exemptions on different properties, that's a red flag
* `F18_OwnerOccupied`:  Is Property Owner Occupied?  
  * ie, it is flagged as a `homestead_exemption`
* `F19_DollarTransfers`:
  * TODO:  real estate transfers for $1?  etc.  (the rtt_summary table)

Other things

* if a certain percentage of a block becomes non-owner occupied?  
* land bank data (parks that are still under land bank)  

* TODO (didn't understand this).  RCO's/multiple RCOs.  Registered Community Organizations.  (cleanings, events).  I you want a zoning variance you have to go to the RCO first.  In stable areas there are more RCOs over time.  
* rental license or vacant property license.  Not all actors get the license.  A history of this on a given property may also be indicative.  helps with triaging owner occupancy.  

* is assessed value radically different from sale price (and it isn't $1 sale)
* some problem owners buy up industrial units and convert them, unsafe
  


### F1_Repairs

Fail or refuse to repair or invest in properties

TODO:  what is the metric/calculation we should use?  

### F2_CodeViolations

Tend to incur code violations

TODO: what is the metric/calculation we should use?

### F3_TaxesAndDebt

Fail to pay Real Estate taxes and/or other city debt such as water bills and code enforcement abatement bills

TODO: what is the metric/calculation we should use?



### F4_CityResources

Use city resources to maintain property.
mowing the lawn
sealing the property
demolishing it

TODO: what is the metric/calculation we should use?

demolitions table
distinct applicantname LIKE 'CITY OF %'
applicanttype = CITY_DEMO
9:10 in recording

### F5_Licenses
Fail to maintain licenses.
vacant property license (non-occupied property)
rental license (occupied property)

TODO: what is the metric/calculation we should use?
wip below
### F6_CorpStructures

Attempt to shield their true identity via complicated corporate structures

TODO: what is the metric/calculation we should use?

### F7_Nuisances

Nuisances within the neighborhood. Either by virtue of how the property is maintained and/or the scope of the properties that they have, that might have minor problems that cumulatively become problematic.

TODO: what is the metric/calculation we should use?


### F8_Insurance

not insuring the property

### F9_Zoning
not using the property for what it is zoned for

### F10_SheriffSalePurchase

buying the property at sheriff's sale and waiting for the neighborhood to turn around.

### F11_AvoidSquatters

actively trying to avoid squatters
### F12_LengthOfOwnership

length of ownership

### F13_HomeValueAppreciation

TODO: from 8/19. Might need to review this one with Philly team. I don't understand the factor.
investor strips all value out of it and not putting anything into it at all and have no intention of doing that. How would we detect/know this?

### F14_BusinessesInArea

varieties of businesses in the area (grocery stores vs bodegas/papis)
vaping stores on every block (drug paraphrenalia)

### F15_NeighborhoodVacantLots

number of vacant lots in neighborhood

### F16_LocalInvestors

in-town vs out-of-town investors (this has been proven to NOT be the latter).






### F17_MultipleHomesteadExemptions and 
### F18_OwnerOccupied

homestead_exemption is amount (0 means NO)

good indicator of whether a property is owner-occupied (which is YES). if LLC has a homestead exemption then it is interesting b/c an LLC with possibly multiple is a legal hook to go after them.

And if an owner has 2 homestead exemptions on different properties, that's a red flag too.


`CASE opa.homestead_exemption WHEN 0 THEN 0 ELSE 1 END AS F18_OwnerOccupied`

In [ ]:
# need the LLC mapping tables first
# 0 AS F17_MultipleHomesteadExemptions

### F19_DollarTransfers

F17_MultipleHomesteadExemptions: Multiple Homestead Exemptions
if an owner has 2 homestead exemptions on different properties, that's a red flag
F18_OwnerOccupied: Is Property Owner Occupied?
ie, it is flagged as a homestead_exemption
F19_DollarTransfers:
TODO: real estate transfers for $1? etc. (the rtt_summary table)

## WIP

Deed transactions...one row per transaction

In [3]:
%%sql 

select 
    rtt.adjusted_assessed_value,
    rtt.adjusted_cash_consideration,
    rtt.adjusted_fair_market_value,
    rtt.adjusted_local_tax_amount,
    rtt.assessed_value,
    rtt.document_date,
    rtt.document_id,
    rtt.document_type,
    rtt.grantees,
    rtt.grantors,
    rtt.legal_remarks,
    rtt.recording_date
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_rtt_summary rtt 
    on opa.parcel_number = rtt.opa_account_num
where opa.location = '2837 KENSINGTON AVE'
order by rtt.document_date desc
;

StatementMeta(, 463061cd-d4d8-4b7b-8796-c73e588b166d, 4, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 12 fields>

Assessments.  One row per assessment

In [5]:
%%sql

select 
    assess.market_value,
    assess.taxable_building,
    assess.taxable_land,
    assess.year,
    assess.exempt_building,
    assess.exempt_land
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_assessments assess
    on opa.parcel_number = assess.parcel_number
where opa.location = '2837 KENSINGTON AVE'
order by assess.year desc
limit 10;

StatementMeta(, 463061cd-d4d8-4b7b-8796-c73e588b166d, 6, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 6 fields>

In [27]:
%%sql

select 
    app.appealgrounds,
    app.appealstatus,
    app.decision,
    app.decisiondate,
    app.opa_owner,
    app.primaryappellant
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_appeals app
    on opa.parcel_number = app.opa_account_num
where opa.location = '2837 KENSINGTON AVE'

StatementMeta(, 349b3e0d-791f-43c4-b325-b37d1aea8b51, 28, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 38 fields>

In [31]:
%%sql 

select 
    cases.casenumber,
    cases.casepriority,
    cases.caseresponsibility,
    cases.casetype,
    cases.investigationcompleted,
    cases.investigationstatus,
    cases.investigationtype
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_case_investigations cases
    on opa.parcel_number = cases.opa_account_num
where opa.location = '2837 KENSINGTON AVE'
order by cases.investigationcompleted desc;

StatementMeta(, 349b3e0d-791f-43c4-b325-b37d1aea8b51, 32, Finished, Available, Finished)

<Spark SQL result set with 20 rows and 33 fields>